In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

##click on account in kaggle and click on API and upload that json

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nabeelfaisal","key":"3d242724e9701f7439f27d45129c20b2"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d tmdb/tmdb-movie-metadata

 56% 5.00M/8.89M [00:00<00:00, 21.3MB/s]
100% 8.89M/8.89M [00:00<00:00, 29.5MB/s]


In [ ]:
ls

kaggle.json  sample_data/  tmdb-movie-metadata.zip


In [ ]:
!unzip tmdb-movie-metadata.zip

Archive:  tmdb-movie-metadata.zip
  inflating: tmdb_5000_credits.csv   
  inflating: tmdb_5000_movies.csv    


In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
df1=pd.read_csv("tmdb_5000_credits.csv")

In [ ]:
df2=pd.read_csv("tmdb_5000_movies.csv")

In [ ]:
df1.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
df2.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [ ]:
df1.columns = ['id','tittle','cast','crew']
df2= df2.merge(df1,on='id')

df2.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,tittle,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


# Average weighting for demographic filtering(based on popularity)

((v/(v+m))*R)+((m/(v+m))*c)------------v=no. of votes, m=min votes required to listed in chart, R-avg rating of movie, c=mean votes across whole report

## we have v(vote count) and R(vote avg) so we can calculte c(mean of votes)

In [ ]:
c=df2['vote_average'].mean()

In [ ]:
print(c)

6.092171559442011


#mean rating of all the movies is 6 out of 10. Now we will find out the value of m

# m= min votes required is more than 90 from movies in terms of voting

In [ ]:
m=df2['vote_count'].quantile(0.9)

In [ ]:
print(m)

1838.4000000000015


## Now create new dataframe of movies which having the vote count more than 90 quantile

In [ ]:
quantile_movies=df2.copy().loc[df2['vote_count']>=m]

In [ ]:
print(quantile_movies)

         budget  ...                                               crew
0     237000000  ...  [{"credit_id": "52fe48009251416c750aca23", "de...
1     300000000  ...  [{"credit_id": "52fe4232c3a36847f800b579", "de...
2     245000000  ...  [{"credit_id": "54805967c3a36829b5002c41", "de...
3     250000000  ...  [{"credit_id": "52fe4781c3a36847f81398c3", "de...
4     260000000  ...  [{"credit_id": "52fe479ac3a36847f813eaa3", "de...
...         ...  ...                                                ...
4291    1200000  ...  [{"credit_id": "52fe4223c3a36847f8006c57", "de...
4300    1200000  ...  [{"credit_id": "52fe424ac3a36847f8012cb9", "de...
4302    1200000  ...  [{"credit_id": "575aee00c3a3682d10000281", "de...
4337    1300000  ...  [{"credit_id": "52fe4218c3a36847f8003803", "de...
4602     350000  ...  [{"credit_id": "52fe423dc3a36847f800ec55", "de...

[481 rows x 23 columns]


In [ ]:
quantile_movies.shape

(481, 23)

In [ ]:
quantile_movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,tittle,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [ ]:
def weight_rating(x,m=m,c=c):
  v=x['vote_count']
  R=x['vote_average']
  return (v/(v+m)*R)+(m/(m+v)*c)

In [ ]:
quantile_movies['score']=quantile_movies.apply(weight_rating,axis=1)

In [ ]:
quantile_movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,tittle,cast,crew,score
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",7.050669


#Now, sort the movies according to weighted_rating and display top 10 movies

In [ ]:
quantile_movies=quantile_movies.sort_values('score',ascending=False)

In [ ]:
quantile_movies[['title','vote_count','vote_average','score']].head(10)

,title,vote_count,vote_average,score
1881,The Shawshank Redemption,8205,8.5,8.059258
662,Fight Club,9413,8.3,7.939256
65,The Dark Knight,12002,8.2,7.920020
3232,Pulp Fiction,8428,8.3,7.904645
96,Inception,13752,8.1,7.863239
3337,The Godfather,5893,8.4,7.851236
95,Interstellar,10867,8.1,7.809479
809,Forrest Gump,7927,8.2,7.803188
329,The Lord of the Rings: The Return of the King,8064,8.1,7.727243
1990,The Empire Strikes Back,5879,8.2,7.697884


In [ ]:
import plotly.express as px

In [ ]:
fig=px.bar((quantile_movies.head(10).sort_values('score',ascending=True)),x='score',y='title',orientation='h')

In [ ]:
fig.show()

#According to demographic filtering we can recommend above 10 movies.

#Content based filtering

In [ ]:
df2[['title','crew','keywords','cast','genres']].head(3)

,title,crew,keywords,cast,genres
0,Avatar,"[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
1,Pirates of the Caribbean: At World's End,"[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""..."
2,Spectre,"[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."


In [ ]:
df2[['title','crew','keywords','cast','genres']].tail(3)

,title,crew,keywords,cast,genres
4800,"Signed, Sealed, Delivered","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam..."
4801,Shanghai Calling,"[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de...",[],"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...",[]
4802,My Date with Drew,"[{""credit_id"": ""58ce021b9251415a390165d9"", ""de...","[{""id"": 1523, ""name"": ""obsession""}, {""id"": 224...","[{""cast_id"": 3, ""character"": ""Herself"", ""credi...","[{""id"": 99, ""name"": ""Documentary""}]"


## most of data is in list of dict format but some data is in string format. so will check what datatype meant to be for the given data

In [ ]:
from ast import literal_eval

In [ ]:
features=df2[['cast','keywords','crew','genres']]

In [ ]:
for feature in features:
  df2[feature]=df2[feature].apply(literal_eval)

In [ ]:
df2.dtypes

budget                    int64
genres                   object
homepage                 object
id                        int64
keywords                 object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
production_countries     object
release_date             object
revenue                   int64
runtime                 float64
spoken_languages         object
status                   object
tagline                  object
title                    object
vote_average            float64
vote_count                int64
tittle                   object
cast                     object
crew                     object
dtype: object

In [ ]:
df2[['title','crew','keywords','cast','genres']].tail(3)

,title,crew,keywords,cast,genres
4800,"Signed, Sealed, Delivered","[{'credit_id': '52fe4df3c3a36847f8275ecf', 'de...","[{'id': 248, 'name': 'date'}, {'id': 699, 'nam...","[{'cast_id': 8, 'character': 'Oliver O’Toole',...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4801,Shanghai Calling,"[{'credit_id': '52fe4ad9c3a368484e16a36b', 'de...",[],"[{'cast_id': 3, 'character': 'Sam', 'credit_id...",[]
4802,My Date with Drew,"[{'credit_id': '58ce021b9251415a390165d9', 'de...","[{'id': 1523, 'name': 'obsession'}, {'id': 224...","[{'cast_id': 3, 'character': 'Herself', 'credi...","[{'id': 99, 'name': 'Documentary'}]"


##find director from movie and add seperate column in dataframe

In [ ]:
def get_director(x):
  for i in x:
    if i['job']=='Director':
      return i['name']
  return np.nan


In [ ]:
df2['director']=df2['crew'].apply(get_director)

In [ ]:
df2.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,tittle,cast,crew,director
0,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de...",James Cameron
1,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://disney.go.com/disneypictures/pirates/,285,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4232c3a36847f800b579', 'de...",Gore Verbinski


## Now convert cast, crew and keywords data into normal list as it is in list of dict format

In [ ]:
def get_list(x):
  if isinstance(x,list):
    names=[i['name'] for i in x]
    return names
  return []


In [ ]:
features=['cast','crew','keywords']
for feature in features:
  df2[feature]=df2[feature].apply(get_list)

In [ ]:
df2[['title','director','cast','crew','keywords']].head(3)

,title,director,cast,crew,keywords
0,Avatar,James Cameron,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Stephen E. Rivkin, Rick Carter, Christopher B...","[culture clash, future, space war, space colon..."
1,Pirates of the Caribbean: At World's End,Gore Verbinski,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Dariusz Wolski, Gore Verbinski, Jerry Bruckhe...","[ocean, drug abuse, exotic island, east india ..."
2,Spectre,Sam Mendes,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Thomas Newman, Sam Mendes, Anna Pinnock, John...","[spy, based on novel, secret agent, sequel, mi..."


##convert value into lowercase

In [ ]:
def clean_data(x):
    if isinstance(x, list):
        return [str(i).replace(" ", "").lower() for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

features = ['cast', 'keywords', 'director', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(clean_data)

In [ ]:
df2['director']

0           jamescameron
1          goreverbinski
2              sammendes
3       christophernolan
4          andrewstanton
              ...       
4798     robertrodriguez
4799         edwardburns
4800          scottsmith
4801          danielhsia
4802     brianherzlinger
Name: director, Length: 4803, dtype: object

In [ ]:
df2['cast']

0       [samworthington, zoesaldana, sigourneyweaver, ...
1       [johnnydepp, orlandobloom, keiraknightley, ste...
2       [danielcraig, christophwaltz, léaseydoux, ralp...
3       [christianbale, michaelcaine, garyoldman, anne...
4       [taylorkitsch, lynncollins, samanthamorton, wi...
                              ...                        
4798    [carlosgallardo, jaimedehoyos, petermarquardt,...
4799    [edwardburns, kerrybishé, marshadietlein, cait...
4800    [ericmabius, kristinbooth, crystallowe, geoffg...
4801    [danielhenney, elizacoupe, billpaxton, alanruc...
4802    [drewbarrymore, brianherzlinger, coreyfeldman,...
Name: cast, Length: 4803, dtype: object

In [ ]:
df2['genres']

0       [{'id':28,'name':'action'}, {'id':12,'name':'a...
1       [{'id':12,'name':'adventure'}, {'id':14,'name'...
2       [{'id':28,'name':'action'}, {'id':12,'name':'a...
3       [{'id':28,'name':'action'}, {'id':80,'name':'c...
4       [{'id':28,'name':'action'}, {'id':12,'name':'a...
                              ...                        
4798    [{'id':28,'name':'action'}, {'id':80,'name':'c...
4799    [{'id':35,'name':'comedy'}, {'id':10749,'name'...
4800    [{'id':35,'name':'comedy'}, {'id':18,'name':'d...
4801                                                   []
4802                     [{'id':99,'name':'documentary'}]
Name: genres, Length: 4803, dtype: object

##generate metadata of movie

In [ ]:
def create_soup(x):
  return ' '.join(x['keywords'])+' '+' '.join(x['cast'])+' '+' '.join(x['director'])+' '+' '.join(x['genres'])

In [ ]:
df2['soup']=df2.apply(create_soup,axis=1)

In [ ]:
df2['soup']

0       cultureclash future spacewar spacecolony socie...
1       ocean drugabuse exoticisland eastindiatradingc...
2       spy basedonnovel secretagent sequel mi6 britis...
3       dccomics crimefighter terrorist secretidentity...
4       basedonnovel mars medallion spacetravel prince...
                              ...                        
4798    unitedstates–mexicobarrier legs arms paperknif...
4799     edwardburns kerrybishé marshadietlein caitlin...
4800    date loveatfirstsight narration investigation ...
4801     danielhenney elizacoupe billpaxton alanruck z...
4802    obsession camcorder crush dreamgirl drewbarrym...
Name: soup, Length: 4803, dtype: object

##remove stopwords and count the number of times word repeated so that we can find similarity between movies

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
count=CountVectorizer(stop_words='english')
count_matrix=count.fit_transform(df2['soup'])

In [ ]:
count_matrix

<4803x65121 sparse matrix of type '<class 'numpy.int64'>'
	with 176687 stored elements in Compressed Sparse Row format>

##create classifier based on similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2=cosine_similarity(count_matrix,count_matrix)

##change the index of the data to movie name

In [ ]:
df2=df2.reset_index()
indices=pd.Series(df2.index,index=df2['title'])

In [ ]:
df2.head(3)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,tittle,cast,crew,director,soup
0,0,237000000,"[{'id':28,'name':'action'}, {'id':12,'name':'a...",http://www.avatarmovie.com/,19995,"[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[samworthington, zoesaldana, sigourneyweaver, ...","[Stephen E. Rivkin, Rick Carter, Christopher B...",jamescameron,cultureclash future spacewar spacecolony socie...
1,1,300000000,"[{'id':12,'name':'adventure'}, {'id':14,'name'...",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drugabuse, exoticisland, eastindiatrad...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[johnnydepp, orlandobloom, keiraknightley, ste...","[Dariusz Wolski, Gore Verbinski, Jerry Bruckhe...",goreverbinski,ocean drugabuse exoticisland eastindiatradingc...
2,2,245000000,"[{'id':28,'name':'action'}, {'id':12,'name':'a...",http://www.sonypictures.com/movies/spectre/,206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[danielcraig, christophwaltz, léaseydoux, ralp...","[Thomas Newman, Sam Mendes, Anna Pinnock, John...",sammendes,spy basedonnovel secretagent sequel mi6 britis...


In [ ]:
def get_recommendations(title,cosine_sim2):
  idx=indices[title]
  sim_scores=list(enumerate(cosine_sim2[idx]))
  sim_scores=sorted(sim_scores,key=lambda x:x[1],reverse=True)
  sim_scores=sim_scores[1:11]
  movie_indices=[i[0] for i in sim_scores]
  return df2['title'].iloc[movie_indices]

##Test it

In [ ]:
get_recommendations('Fight Club',cosine_sim2)

4564    Straight Out of Brooklyn
4247       Me You and Five Bucks
4638    Amidst the Devil's Wings
4462          The Young Unknowns
4574                    Roadside
2723     The Caveman's Valentine
45                   World War Z
3043            End of the Spear
4589                      Fabled
4426                 A Fine Step
Name: title, dtype: object

In [ ]:
get_recommendations('The Godfather',cosine_sim2)

2731      The Godfather: Part II
867      The Godfather: Part III
4638    Amidst the Devil's Wings
4564    Straight Out of Brooklyn
4370                Brigham City
2507                   Slow Burn
3293                 10th & Wolf
4205                    N-Secure
4488                  Adulterers
4780                 Dutch Kills
Name: title, dtype: object

#download movies in csv format

In [ ]:
from google.colab import files

In [ ]:
df2.to_csv('movies.csv')

In [ ]:
files.download('movies.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df2.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'tittle', 'cast', 'crew', 'director',
       'soup'],
      dtype='object')